<a href="https://colab.research.google.com/github/thedstrat/Machine-Learning-Python-Projects/blob/master/Collaborative_Filtering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Collaborative Filtering Using Fastai and the Jester Database

Part 1: 

In [0]:
# Mount your Google Drive which contains the Jester database
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from fastai.collab import *
from fastai.tabular import *
import pandas as pd
import numpy as np

In [0]:
user, item, title = 'userID','SongId','title'

In [0]:
#Specify directory of the Jester file
path = "/content/drive/My Drive/documents/songsDataset.csv"

Song Database Notes:

---

1.  Retrieved from https://www.kaggle.com/rymnikski/dataset-for-collaborative-filters/downloads/dataset-for-collaborative-filters.zip/1





In [0]:
songdata = pd.read_csv(path)

print(songdata.head())

   userID  SongID  Rating
0       0    7171       5
1       0    8637       4
2       0   21966       4
3       0   35821       5
4       0   82446       5


In [0]:
# Prep the ratings df for the CNN (split into test and training sets)
data = CollabDataBunch.from_df(songdata, valid_pct=0.2) 

In [0]:
y_range = (1, 5) #set the rating value range

##Dot Bias Model

In [0]:
#Train the classifier (Dot Product Model)
learn = collab_learner(data, n_factors=40, y_range=y_range, wd = .1) #What's n_factors?

In [0]:
#Find the optimal learning rate
learn.lr_find()
learn.recorder.plot()

#I'm getting very different lr rate graphs. Read up on this again.

In [0]:
learn.unfreeze() #unfreezing to improve accuracy
learn.fit_one_cycle(1) #3 epochs

epoch,train_loss,valid_loss,time
0,2.389686,2.413594,01:42
1,2.352038,2.374702,01:42
2,2.277334,2.346240,01:43


In [0]:
#Save the weights and learner object
learn.save('/content/drive/My Drive/Colab Notebooks/Saved Models/trained_model_songs1', return_path = True)
learn.export('/content/drive/My Drive/Colab Notebooks/Saved Models/songs_model.pkl')

## Embeddings Model

In [0]:
#Embeddings Model
learn2 = collab_learner(data, use_nn=True, emb_szs={'userID': 40, 'SongId':40}, layers=[256, 128], y_range=(1, 5)) #Make embedding with 40 rows for userID and 40 columns for songID I believe

In [0]:
learn.lr_find() 
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(4, 1e-5)

epoch,train_loss,valid_loss,time
0,2.226114,2.345938,01:41
1,2.232084,2.345295,01:42
2,2.211262,2.344895,01:42
3,2.249335,2.344831,01:41


In [0]:
#Save the weights and learner object
learn.save('/content/drive/My Drive/Colab Notebooks/Saved Models/trained_model_songs2', return_path = True)
learn.export('/content/drive/My Drive/Colab Notebooks/Saved Models/songs_model2.pkl')

## Analysis

In [0]:
#Get Top 1000 Songs
g = songdata.groupby('SongID')['Rating'].count()
top_songs = g.sort_values(ascending=False).index.values[:1000]
top_songs = top_songs.astype(str)